In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os 
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [2]:
#Predicting Google's stock price
dataset_train = pd.read_csv('Google_Stock_Price_Train.csv')
training_set = dataset_train.iloc[:, 1:2].values

In [3]:
#Feature Scaling
#Normalising it
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))
training_set_scaled = sc.fit_transform(training_set)

In [4]:
#creating a structure with 60 timesteps and 1 output. Looking at 60 previous values
#X-train will contain 60 stock prices. y_train will contain 1 day after that
X_train = []
y_train = []
for i in range(60,1258):
    X_train.append(training_set_scaled[i-60:i,0])
    y_train.append(training_set_scaled[i,0])
X_train, y_train = np.array(X_train), np.array(y_train)

In [5]:
#Reshaping it
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))#adding a dimension for indicator

In [6]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [7]:
#Initialising RNN
regressor = Sequential()

In [8]:
# Adding first lstm layer and regularisation(Dropout)

regressor.add(LSTM(units = 50, return_sequences=True, input_shape=(X_train.shape[1], 1))) #return_sequences set to true for stacked LSTM layers
regressor.add(Dropout(0.2)) #Dropping 20% neurons in the layers

In [9]:
#Adding second LSTM Layer with regularisation(Dropout)
regressor.add(LSTM(units = 50, return_sequences=True)) #No need to add input layer since units automatically tells 
regressor.add(Dropout(0.2))

In [10]:
#Adding third LSTM layer
regressor.add(LSTM(units = 50, return_sequences=True))
regressor.add(Dropout(0.2))

In [11]:
#Adding fourth LSTM layer
regressor.add(LSTM(units = 50)) #return_sequences to be removed since we are not returning anything else
regressor.add(Dropout(0.2))

In [12]:
#Op layer
regressor.add(Dense(units=1))

In [13]:
#Compile
regressor.compile(optimizer='adam',loss='mean_squared_error')#can use rmsprop also as optimizer

In [14]:
#Fitting the RNN. Usually cry at this point due to my PC
regressor.fit(X_train,y_train,batch_size=32,epochs=100)

Epoch 1/100
38/38 [==============================] - 4s 105ms/step - loss: 0.0457
Epoch 2/100
38/38 [==============================] - 4s 110ms/step - loss: 0.0070
Epoch 3/100
38/38 [==============================] - 4s 109ms/step - loss: 0.0058
Epoch 4/100
38/38 [==============================] - 4s 110ms/step - loss: 0.0058
Epoch 5/100
38/38 [==============================] - 4s 111ms/step - loss: 0.0056
Epoch 6/100
38/38 [==============================] - 5s 128ms/step - loss: 0.0053
Epoch 7/100
38/38 [==============================] - 4s 110ms/step - loss: 0.0050
Epoch 8/100
38/38 [==============================] - 4s 103ms/step - loss: 0.0049
Epoch 9/100
38/38 [==============================] - 4s 107ms/step - loss: 0.0046
Epoch 10/100
38/38 [==============================] - 4s 110ms/step - loss: 0.0046
Epoch 11/100
38/38 [==============================] - 5s 127ms/step - loss: 0.0043
Epoch 12/100
38/38 [==============================] - 4s 115ms/step - loss: 0.0039
Epoch 13/100


38/38 [==============================] - 4s 107ms/step - loss: 0.0014
Epoch 100/100
38/38 [==============================] - 4s 108ms/step - loss: 0.0014


In [16]:
#Getting the test set
dataset_test = pd.read_csv('Google_Stock_Price_Test.csv')
real_stock_price = dataset_test.iloc[:, 1:2].values

In [ ]:
#Getting predicted stock price
dataset_total = pd.concat((dataset_train['Open'],dataset_test['Open']), axis=0)
inputs = dataset_total[len(dataset_total) - len(dataset_test) -60:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)